In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(
    credential=DefaultAzureCredential(),
    path="config.json",
)

Found the config file in: config.json


In [7]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name = "furlanp-test-model"

model = Model(
    path="model.joblib",
    type=AssetTypes.CUSTOM_MODEL,
    name=model_name,
    version="1",
)

ml_client.models.create_or_update(model)

Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'furlanp-test-model', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/aceb381b-74dc-4f6d-b21f-c0b527db1729/resourceGroups/rg-ki-ds-s/providers/Microsoft.MachineLearningServices/workspaces/mlw-ki-ds-s/models/furlanp-test-model/versions/1', 'Resource__source_path': '', 'base_path': '/Users/FurlanP/Documents/Repos/furlanp-azure-test', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x12715e870>, 'serialize': <msrest.serialization.Serializer object at 0x12715ea80>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/aceb381b-74dc-4f6d-b21f-c0b527db1729/resourceGroups/rg-ki-ds-s/workspaces/mlw-ki-ds-s/datastores/workspaceblobstore/paths/LocalUpload/211b913756b6a90a4eea88203b115712/model.joblib', 'datastore': None, 'utc_time_created': None, 'flavors

In [8]:
from azure.ai.ml.entities import Environment

env_name = "furlanp-test-env"

env = Environment(
    name=env_name,
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    version="1",
)

ml_client.environments.create_or_update(env)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'furlanp-test-env', 'description': None, 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/aceb381b-74dc-4f6d-b21f-c0b527db1729/resourceGroups/rg-ki-ds-s/providers/Microsoft.MachineLearningServices/workspaces/mlw-ki-ds-s/environments/furlanp-test-env/versions/1', 'Resource__source_path': '', 'base_path': '/Users/FurlanP/Documents/Repos/furlanp-azure-test', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x12715fd10>, 'serialize': <msrest.serialization.Serializer object at 0x12715e840>, 'version': '1', 'conda_file': {'channels': ['defaults'], 'dependencies': ['python=3.11', 'pip', {'pip': ['pandas', 'numpy', 'scikit-learn', 'joblib', 'azureml-inference-s

#### Change image build to compute!
By default azure uses serverless to build environment images. \
We need to change that to one of the compute nodes (must be CPU) from the workspace. 

For example:

az login \
az ml workspace update --name mlw-ki-ds-s --resource-group rg-ki-ds-s --image-build-compute FurlanP2

In [9]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

endpoint_name = "furlanp-test-endpoint"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="furlanp-test-endpoint",
    auth_mode="key",
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [10]:
from azure.ai.ml.entities import ManagedOnlineDeployment
from azure.ai.ml.entities import CodeConfiguration

model = ml_client.models.get(name=model_name, version=1)
env = ml_client.environments.get(name=env_name, version=1)

deployment_name = "furlanp-test-deployment"

deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(".", "score.py"), 
    egress_public_network_access="disabled",
)

deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint furlanp-test-endpoint exists
Uploading furlanp-azure-test (0.07 MBs): 100%|██████████| 71735/71735 [00:00<00:00, 140269.58it/s]




........................................................................................................

In [12]:
import json
import numpy as np

res = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    request_file="./sample_data.json",
)

res = json.loads(res)
res = np.array(res)
print(res.shape) # (2,15)

(2, 15)
